# PART A

In [1]:
# imports
import os
import time
import glob
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import itertools
import wandb
from env import create_standard_grid, create_four_room

# Optional: configure matplotlib for notebooks
%matplotlib inline


### Create environment

In [3]:
all_configs = []

# -------------------------------
# 10x10 Grid World: Q-learning (8)
# -------------------------------
for tp in [0.7, 1.0]:
    for start_state in [(0, 4), (3, 6)]:
        for strat in ['eps_greedy', 'softmax']:
            name = f"std_q_tp{tp}_ss{start_state}_strat{strat}"
            cfg = {
                'name': name,
                'alg': 'q_learning',
                'env_builder': create_standard_grid,
                'env_kwargs': {
                    # ✅ FIX: convert tuple to 2D numpy array
                    'start_state': np.array([[start_state[0], start_state[1]]]),
                    'transition_prob': tp,
                    'wind': False
                },
                'strategy': strat
            }
            all_configs.append(cfg)

# -------------------------------
# 10x10 Grid World: SARSA (8)
# -------------------------------
for wind in [True, False]:
    for start_state in [(0, 4), (3, 6)]:
        for strat in ['eps_greedy', 'softmax']:
            name = f"std_sarsa_wind{wind}_ss{start_state}_strat{strat}"
            cfg = {
                'name': name,
                'alg': 'sarsa',
                'env_builder': create_standard_grid,
                'env_kwargs': {
                    # ✅ FIX here too
                    'start_state': np.array([[start_state[0], start_state[1]]]),
                    'transition_prob': 1.0,
                    'wind': wind
                },
                'strategy': strat
            }
            all_configs.append(cfg)

# -------------------------------
# Four-Room configs (no fix needed)
# -------------------------------
for goal_change in [True, False]:
    all_configs.append({
        'name': f"four_q_goalchange{goal_change}_strateps_greedy",
        'alg': 'q_learning',
        'env_builder': create_four_room,
        'env_kwargs': {'goal_change': goal_change},
        'strategy': 'eps_greedy'
    })
for goal_change in [True, False]:
    all_configs.append({
        'name': f"four_sarsa_goalchange{goal_change}_strateps_greedy",
        'alg': 'sarsa',
        'env_builder': create_four_room,
        'env_kwargs': {'goal_change': goal_change},
        'strategy': 'eps_greedy'
    })

### Functions

In [8]:
# Safe reward scalar extractor for 1-element arrays
def reward_scalar(r):
    return float(np.ravel(r)[0])

# epsilon greedy function
# Chose to use correct datatype outputs as it becomes a problem in downstream tasks.

def epsilon_greedy_action(Q: np.ndarray, state: int, epsilon: float) -> int:
    """Pick epsilon-greedy action from Q[state]."""
    if np.random.rand() < epsilon:
        return np.random.randint(Q.shape[1])
    return int(np.argmax(Q[state]))

# --- Episode termination check ---
def is_terminal(state: int, goal_states_seq: np.ndarray) -> bool:
    return state in set(map(int, np.array(goal_states_seq).flatten()))

# --- Rollouts for evaluation ---
def evaluate_policy(env, Q: np.ndarray, episodes: int = 20, max_steps: int = 100) -> float:
    total = 0.0
    for _ in range(episodes):
        s = env.reset()
        ep_ret = 0.0
        for _ in range(max_steps):
            a = int(np.argmax(Q[s]))
            s_next, r = env.step(s, a)
            ep_ret += reward_scalar(r)
            s = int(s_next)
            if is_terminal(s, env.goal_states_seq):
                break
        total += ep_ret
    return total / episodes
    
def softmax_action(Q, state, tau):
    tau = float(tau)
    if tau <= 0:
        return int(np.argmax(Q[state]))
    z = Q[state] / tau
    z = z - np.max(z)
    p = np.exp(z)
    p = p / np.sum(p)
    return int(np.random.choice(len(p), p=p))


def select_action(Q, state, strategy, param):
    if strategy == 'eps_greedy':
        if np.random.rand() < float(param):
            return int(np.random.randint(Q.shape[1]))
        return int(np.argmax(Q[state]))
    elif strategy == 'softmax':
        return softmax_action(Q, state, float(param))
    else:
        raise ValueError('Unknown exploration strategy: ' + str(strategy))

### Q-learning function

In [6]:
def train_q_learning_fixed(env, episodes, alpha, gamma, strategy, param, max_steps=100, run=None, seed=None):
    """
    Q-Learning with optional WandB logging.
    """
    n_states, n_actions = env.num_states, env.num_actions
    Q = np.zeros((n_states, n_actions))
    returns = []

    for ep in range(episodes):
        state = int(env.reset())
        total_reward = 0.0

        for _ in range(max_steps):
            a = select_action(Q, state, strategy, param)
            s_next, r = env.step(state, a)
            r = reward_scalar(r)
            total_reward += r
            s_next = int(s_next)

            if is_terminal(s_next, env.goal_states_seq):
                Q[state, a] += alpha * (r - Q[state, a])
                break

            td_target = r + gamma * np.max(Q[s_next])
            Q[state, a] += alpha * (td_target - Q[state, a])
            state = s_next

        returns.append(total_reward)

        # ✅ WandB logging
        if run is not None:
            wandb.log({
                "episode": ep,
                "episode_return": total_reward,
                "algorithm": "Q-learning",
                "alpha": alpha,
                "gamma": gamma,
                "param": param,
                "strategy": strategy,
                "seed": seed
            })

        if (ep + 1) % 200 == 0:
            print(f"[Q] Episode {ep+1}/{episodes} | Mean(Last50): {np.mean(returns[-50:]):.2f}")

    return Q, returns


### SARSA function

In [7]:
def train_sarsa_fixed(env, episodes, alpha, gamma, strategy, param, max_steps=100, run=None, seed=None):
    """
    SARSA with optional WandB logging.
    """
    n_states, n_actions = env.num_states, env.num_actions
    Q = np.zeros((n_states, n_actions))
    returns = []

    for ep in range(episodes):
        s = int(env.reset())
        a = select_action(Q, s, strategy, param)
        total_reward = 0.0

        for _ in range(max_steps):
            s_next, r = env.step(s, a)
            r = reward_scalar(r)
            total_reward += r
            s_next = int(s_next)

            if is_terminal(s_next, env.goal_states_seq):
                Q[s, a] += alpha * (r - Q[s, a])
                break

            a_next = select_action(Q, s_next, strategy, param)
            td_target = r + gamma * Q[s_next, a_next]
            Q[s, a] += alpha * (td_target - Q[s, a])
            s, a = s_next, a_next

        returns.append(total_reward)

        # ✅ WandB logging
        if run is not None:
            wandb.log({
                "episode": ep,
                "episode_return": total_reward,
                "algorithm": "SARSA",
                "alpha": alpha,
                "gamma": gamma,
                "param": param,
                "strategy": strategy,
                "seed": seed
            })

        if (ep + 1) % 200 == 0:
            print(f"[SARSA] Episode {ep+1}/{episodes} | Mean(Last50): {np.mean(returns[-50:]):.2f}")

    return Q, returns


### Setting up Wandb

In [4]:
wandb.init()

# 
sweep_config_1 = {
    'method': 'grid',   # try every combination
    'metric': {
        'name': 'mean_eval_return',
        'goal': 'maximize'
    },
    'parameters': {
        'alpha': {'values': [0.001, 0.01, 0.1, 1.0]},
        'gamma': {'values': [0.7, 0.8, 0.9, 1.0]},
        'param': {'values': [0.001, 0.01, 0.05, 0.1]},  # Use param if your code expects epsilon as "param"
        'algorithm': {'values': ['q_learning']},
        'strategy': {'values': ['eps_greedy']},
        'env_name': {'values': ['std_q_tp0.7_ss(0,4)_strateps_greedy']}
    }
}

wandb: Currently logged in as: leenh050418 (leenh050418-iit-madras-foundation) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Exception in thread IntMsgThr:
Traceback (most recent call last):
  File "c:\Users\Kutral\AppData\Local\Programs\Python\Python311\Lib\threading.py", line 1038, in _bootstrap_inner
    self.run()
  File "C:\Users\Kutral\AppData\Roaming\Python\Python311\site-packages\ipykernel\ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "c:\Users\Kutral\AppData\Local\Programs\Python\Python311\Lib\threading.py", line 975, in run
    self._target(*self._args, **self._kwargs)
  File "c:\Users\Kutral\AppData\Local\Programs\Python\Python311\Lib\site-packages\wandb\sdk\wandb_run.py", line 333, in check_internal_messages
    self._loop_check_status(
  File "c:\Users\Kutral\AppData\Local\Programs\Python\Python311\Lib\site-packages\wandb\sdk\wandb_run.py", line 236, in _loop_check_status
    local_handle = request()
                   ^^^^^^^^^
  File "c:\Users\Kutral\AppData\Local\Programs\Python\Python311\Lib\site-packages\wandb\sdk\interface\interface.py", line 1022, in delive

In [5]:
sweep_id = wandb.sweep(sweep_config_1, project="GPA_2_PART_A")
print("Sweep ID:", sweep_id)


Create sweep with ID: umdu1k6v
Sweep URL: https://wandb.ai/leenh050418-iit-madras-foundation/GPA_2_PART_A/sweeps/umdu1k6v
Sweep ID: umdu1k6v


In [9]:
def train_one_sweep_run():
    wandb.init()
    cfg = wandb.config

    # setup environment
    env = create_standard_grid(transition_prob=0.7, start_state=np.array([[0,4]]))

    # pick the correct algorithm
    if cfg.algorithm == "q_learning":
        Q, returns = train_q_learning_fixed(
            env,
            episodes=1500,
            alpha=cfg.alpha,
            gamma=cfg.gamma,
            strategy=cfg.strategy,
            param=cfg.param,
            max_steps=100,
            run=wandb.run
        )
    else:
        Q, returns = train_sarsa_fixed(
            env,
            episodes=1500,
            alpha=cfg.alpha,
            gamma=cfg.gamma,
            strategy=cfg.strategy,
            param=cfg.param,
            max_steps=100,
            run=wandb.run
        )

    # Evaluate
    avg_eval = evaluate_policy(env, Q, episodes=20, max_steps=100)

    wandb.log({
        'mean_eval_return': avg_eval,
        'final_mean_reward_last50': np.mean(returns[-50:])
    })

    print(f"Run finished | α={cfg.alpha}, γ={cfg.gamma}, param={cfg.param} → avg_eval={avg_eval:.2f}")

In [10]:
wandb.agent(sweep_id, function=train_one_sweep_run, count=None)


wandb: WARNING Calling wandb.login() after wandb.init() has no effect.
wandb: Agent Starting Run: ukomeosi with config:
wandb: 	algorithm: q_learning
wandb: 	alpha: 0.001
wandb: 	env_name: std_q_tp0.7_ss(0,4)_strateps_greedy
wandb: 	gamma: 0.7
wandb: 	param: 0.001
wandb: 	strategy: eps_greedy


[Q] Episode 200/1500 | Mean(Last50): -116.94
[Q] Episode 400/1500 | Mean(Last50): -113.38
[Q] Episode 600/1500 | Mean(Last50): -112.16
[Q] Episode 800/1500 | Mean(Last50): -113.00
[Q] Episode 1000/1500 | Mean(Last50): -107.60
[Q] Episode 1200/1500 | Mean(Last50): -111.10
[Q] Episode 1400/1500 | Mean(Last50): -107.98
Run finished | α=0.001, γ=0.7, param=0.001 → avg_eval=-71.50


alpha,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
episode,▁▁▁▂▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▄▄▄▅▅▅▅▆▆▆▆▆▇▇▇████
episode_return,▅▇▆▇▁▇▆▇▇▅▇▆▇▆▁█▇▇▇▇▇▇▆▇▆▇▇▇▇▇▇▇▇▇▇▆▆▇▆▁
final_mean_reward_last50,▁
gamma,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
mean_eval_return,▁
param,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
algorithm,Q-learning
alpha,0.001
episode,1499
episode_return,-100


wandb: Agent Starting Run: eu14tnfb with config:
wandb: 	algorithm: q_learning
wandb: 	alpha: 0.001
wandb: 	env_name: std_q_tp0.7_ss(0,4)_strateps_greedy
wandb: 	gamma: 0.7
wandb: 	param: 0.01
wandb: 	strategy: eps_greedy


[Q] Episode 200/1500 | Mean(Last50): -115.16
[Q] Episode 400/1500 | Mean(Last50): -108.74
[Q] Episode 600/1500 | Mean(Last50): -108.06
[Q] Episode 800/1500 | Mean(Last50): -109.44
[Q] Episode 1000/1500 | Mean(Last50): -111.20
[Q] Episode 1200/1500 | Mean(Last50): -103.92
[Q] Episode 1400/1500 | Mean(Last50): -103.26
Run finished | α=0.001, γ=0.7, param=0.01 → avg_eval=-100.00


...

[Q] Episode 200/1500 | Mean(Last50): -70.48
[Q] Episode 400/1500 | Mean(Last50): -67.44
[Q] Episode 600/1500 | Mean(Last50): -56.10
[Q] Episode 800/1500 | Mean(Last50): -68.96
[Q] Episode 1000/1500 | Mean(Last50): -84.98
[Q] Episode 1200/1500 | Mean(Last50): -62.56
[Q] Episode 1400/1500 | Mean(Last50): -52.06
Run finished | α=1, γ=1, param=0.1 → avg_eval=-77.85


alpha,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
episode,▁▁▁▁▁▂▂▂▃▃▃▃▃▃▃▃▃▃▄▄▄▄▄▅▅▅▆▆▆▆▇▇▇▇▇▇▇███
episode_return,▇▁▃▆▆▇▆▆█▇█▇▆▃▇▆▆▇▄█▆▇▆▇▇█▆▆▇▇█▆▇█▇▇██▅▇
final_mean_reward_last50,▁
gamma,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
mean_eval_return,▁
param,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
algorithm,Q-learning
alpha,1
episode,1499
episode_return,-59


wandb: Sweep Agent: Waiting for job.
wandb: Sweep Agent: Exiting.


Error in callback <bound method _WandbInit._post_run_cell_hook of <wandb.sdk.wandb_init._WandbInit object at 0x00000296B1874890>> (for post_run_cell), with arguments args (<ExecutionResult object at 296b188cc10, execution_count=10 error_before_exec=None error_in_exec=None info=<ExecutionInfo object at 296b186c490, raw_cell="wandb.agent(sweep_id, function=train_one_sweep_run.." store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell:/d%3A/...SEM5/RL/GPA_2_new/Grid-World-Environment-main/PART_A.ipynb#X22sZmlsZQ%3D%3D> result=None>,),kwargs {}:


AlreadyJoinedError: 